# Основная функция ASR #

- Импортируются необходимые библиотеки: speech_recognition, os, pydub, silence из pydub и glob.
- Создается объект распознавания речи.
- Создается функция get_large_audio_transcription, которая разбивает аудиофайл на куски и применяет к каждому куску распознавание речи.
- Аудиофайл открывается с помощью pydub, затем разбивается на куски в местах, где длина тишины равна или больше 200 миллисекунд, и полученные куски сохраняются в переменной chunks.
- Создается объект silence с длиной тишины 1500 миллисекунд.
- Создается папка "audio-chunks" для сохранения полученных кусков.
- Куски, длина которых меньше 3 секунд, объединяются в один кусок.
- К каждому куску добавляется тишина длиной 1500 миллисекунд.
- Каждый кусок экспортируется в формате WAV и сохраняется в папку "audio-chunks".
- Каждый экспортированный кусок аудио распознается с помощью объекта Recognizer и сохраняется в переменной text.
- Если распознавание не удалось из-за неизвестного значения ошибки, выводится сообщение об ошибке.
- Если распознавание удалось, то полученный текст приводится к формату заглавной буквы в начале предложения и сохраняется в переменную whole_text.
- Экспортированный кусок аудио сохраняется в папку, заданную в пути os.path.join().
- Возвращается список whole_text с распознанным текстом и счетчик cnt, который увеличивается на единицу для каждого сохраненного куска аудио.

In [1]:
# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
import glob

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path,cnt):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_mp3(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,min_silence_len=200, silence_thresh=-40) 
    silence = AudioSegment.silent(duration=1500)
    folder_name = "audio-chunks"
    target_length = 3*1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk 
        else:
            output_chunks.append(chunk)
    for i in range(len(output_chunks)):
        output_chunks[i] = output_chunks[i] + silence
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    else: 
        files = glob.glob(folder_name + '/*')
        for f in files:
            os.remove(f)
    whole_text = []
    
    # process each chunk 
    for i, audio_chunk in enumerate(output_chunks):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened,language="ru-RU")
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text.append(text)
                audio_chunk.export(os.path.join(r'C:\Users\wwwjo\Desktop\SII\DIP_lom\data', f"chunk{cnt}.wav"), format="wav")
                cnt += 1
    # return the text for all chunks detected
    return whole_text, cnt

# Запуск мультифайлового распознавания речи #
- Объявляется переменная cnt со значением 0, которая будет использоваться для создания уникальных имен файлов.
- С помощью функции glob.glob() выбираются все файлы в указанной папке r'C:\Users\wwwjo\Desktop\SII\DIP_lom\input_data\opergamer\'.
- Создаются переменные epoch и transcriptions. Переменная epoch не используется в данном коде. Переменная transcriptions будет использоваться для хранения всех полученных транскрипций.
- Начинается цикл for, который проходится по всем файлам, выбранным в шаге 2.
- Вызывается функция get_large_audio_transcription(), которая принимает путь к аудио файлу и текущее значение cnt. Функция разбивает аудио файл на чанки и применяет к каждому чанку распознавание речи с помощью Google Speech API. Также сохраняются полученные транскрипции в переменной whole_text.
- После выполнения функции get_large_audio_transcription(), все полученные транскрипции добавляются в список transcriptions.
- Печатается текущее значение cnt, чтобы отслеживать количество созданных файлов.


In [ ]:
cnt = 0
files = glob.glob(r'C:\Users\wwwjo\Desktop\SII\DIP_lom\input_data\opergamer\*') # выбор необходимой папки с файлами для создания
epoch = 0
transcriptions = []
for f in files:
    # epoch +=1
    # print(f)
    # if epoch >2: 
    #     break
    trans, cnt = get_large_audio_transcription(f,cnt)
    for i in trans:
        transcriptions.append(i)
    print(cnt)

# Создание датасета на основе аудиосегментов с транскрипцией #
- Импортируется библиотека Pandas для работы с таблицами данных.
- Создается пустой объект DataFrame с двумя столбцами 'Name' и 'Transcript'.
- В цикле проходятся все полученные транскрипты, и для каждого транскрипта создается новая строка в объекте DataFrame, где в столбец 'Name' записывается имя файла в формате "data{i}", а в столбец 'Transcript' записывается соответствующий транскрипт.
- Полученный DataFrame экспортируется в Excel-файл с помощью метода to_excel(), указывая путь к файлу, отключение индекса и наличие заголовка только на первой строке.

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['Name', 'Transcript'])
for i in range(len(transcriptions)):
    df = df.append({"Name": 'data{0}'.format(i),'Transcript': transcriptions[i]},ignore_index=True)
df.to_excel(r'C:\Users\wwwjo\Desktop\SII\DIP_lom\DATASET.xlsx', index=False, header=1)

# Нормализация транскрипций #
- Импортируется библиотека transliterate, которая используется для транслитерации кириллических символов на латинские.
- В цикле проходится по каждой строке датафрейма и производится транслитерация русских слов на латиницу с помощью функции translit из библиотеки transliterate.
- Заменяются некоторые латинские символы на русские эквиваленты с помощью метода replace.
- Измененный датафрейм сохраняется в том же Excel-файле.

In [ ]:
import transliterate
from transliterate import translit
for i in range(len(df)):
    text = df['Transcript'][i]
    text = translit(df['Transcript'][i], 'ru')
    text = text.replace('w','в')
    text = text.replace('W','В')
    text = text.replace('q','ку')
    text = text.replace('Q','Ку')
    df['Transcript'][i] = text
df.to_excel(r'C:\Users\wwwjo\Desktop\SII\DIP_lom\DATASET.xlsx', index=False, header=1)